# Laravel 2 : Blade


## Introduction

`Blade` est un moteur de rendu simple, mais puissant. En tant que tel, il sert à crer des gabarits d'affichage. Il se rapproche beaucoup d'autres moteurs comme **Twig** (Symfony) ou **Jinja** (Django). Tous les modèles Blade sont compilés en code PHP et mis en cache jusqu'à ce qu'ils soient modifiés, ce qui signifie que Blade n'ajoute pratiquement aucune surcharge à l'application. 

Les fichiers des modèles Blade utilisent l'extension de fichier `.blade.php` et sont généralement stockés dans le répertoire `resources/views`.

Les vues Blade peuvent être renvoyées depuis les routes ou les contrôleurs en utilisant la fonction de support `view` du routeur.

In [ ]:
%%php 

Route::get('/greeting', function () {
    return view('greeting', ['name' => 'FINN']);
});

Dans l'exemple ci-dessus, la route fait directement appel à une fonction anonyme qui rend une **vue** correpondant à un fichier `greeting.blade.php`, auquel est transmis un tableau de données.

Le fichier en question pourrait tout simplement ressembler à ceci :

In [ ]:
%%html

<html>
  <head></head>
  <body>
    <h1>Hello {{ substr(ucfirst($name),0,1) }} !</h1>
  </body>
</html>

Comme on le voit, un emplacement est ménagé, marqué par les doubles accolades, pour insérer la valeur d'une variable du nom de `name`, dont la valeur est stipulée dans le tableau associatif de la fonction anonyme.

De surcroît, cette syntaxe permet d'inclure dans le gabarit Blade du code PHP arbitraire, comme :

In [ ]:
Hello {{ $name }}, il est {{ time() }}

## Directives Blade

Blade offre un certain nombre de « _directives_ », qui permettent d'intervenir sur le comportement des gabarits.

### Conditionnelles

Faire de l'affichage conditionnel :

In [ ]:
%%blade

@if (count($records) === 1)
    J’ai un enregistrement !
@elseif (count($records) > 1)
    J’ai plusieurs enregistrements !
@else
    Je n’ai aucun enregistrement !
@endif

Blade admet également les conditions `@unless`, `@isset` et `@empty`.

On peut également utiliser la structure de contrôle `@switch` qui fonctionne de la même manière qu'en PHP, avec une suite de `@case`.

### Exercice

On passe à un gabarit Blade le nom d'une personne, la date du jour, un nombre de tâches en cours et un nombre de taches terminées.
- On veut affiche le nom de la personne
- on veut afficher le nombre total de tâches
- s'il reste des tâches en cours, on affiche un message , le texte en couleur orange
- Si toutes les tâches sont terminées , on affiche un autre message et le texte en vert.

### Itérateurs

Blade sait itérer sur des listes de valeurs, avec diverses syntaxes de boucles :

In [ ]:
%%blade 

@for ($i = 0; $i < 10; $i++)
     <p>{{ $i }}</p>
@endfor
 
@foreach ($users as $user)
    <p>Nom : {{ $user->name }}</p>
@endforeach
 
@forelse ($users as $user)
    <li>Nom : {{ $user->name }}</li>
@empty
    <p>No users</p>
@endforelse
 
@while (true)
    <p>Oups ! Une boucle infinie.</p>@endwhile

## Exercice

On veut afficher une liste de tâches, liste qui est définie dans une variable globale `taskList`.
- l'affichage de fait sous forme de liste HTML
- chaque tâche est associée à une date de fin.
- on voudra compter le nombre de tâches en cours et de tâches terminéeset afficher ces nombres


Les itérateurs Blade possèdent en particulier un variable spéciale, `$loop`, qui permet d'accéder à l'itérateur lui-même. `$loop` est un objet dont lespropriétés sont :

| Propriété | Description |
|---|---|
| index | L'indice courant de la boucle (commence à 0 |
| iteration | L'itération courante de la boucle (commence à 1) |
| remaining | Le nombre d'itérations restantes |
| count | Le nombre total d'itérations |
| first | Est-ce la première itération ? |
| last | Est-ce la dernière itération ? |
| even | Est-ce une itération paire ? |
| odd | Est-ce une itération impaire ? |
| depth | Le niveau d'imbrication de la boucle courante |
| parent | La boucle du niveau immédiatement supérieur |

### Autres directives

Blade dispose de nombreuses autres directives dont voici quelques exemples :
- `@auth` : l'utilisateur est authentifié
- `@guest` : l'utilisateur est un invité
- `@env` : détermine l'environnement de production ('development', 'staging', 'production', etc.)
- `@hasSection` : détermine si une vue hiérarchique à une section
- `@sectionMissing` : propose un contenu de remplacement si une section est manquante dans une vue hérarchique
- `@php` : permet d'inclure du code PHP arbitraire
    
Des directivs peuvent modifier des éléments HTML, comme
- `@class` : ajoute des classes CSS conditionnelles à un élément du gabarit    

In [ ]:
%%html

@php
    $isActive = false;
    $hasError = true;
@endphp
 
<span @class([
    'p-4',
    'font-bold' => $isActive,
    'text-gray-500' => ! $isActive,
    'bg-red' => $hasError,
])></span>

- `@style` : idem mais avec l'attribut `style`
- `@checked`, `@disabled`, `@readonly`, `@required` : ajoutent, sous condition, les attributs correspondants à un élément HTML

In [ ]:
%%html

<input type="email" name="email" value="email@laravel.com" @readonly($user->isNotAdmin()) />

<div class="alert alert-block alert-info">
<b>Note :</b> Il est possible, en plus de directives natives, de créer ses propres directives
</div>

## Vue hiérarchiques

Le principal intérêt d'utiliser **Blade** plutôt que PHP, c'est de pouvoir **composer** des vues à partir de différents fragments. Ce serait possible de le faire en PHP grâce à des `include` ou `require`, mais le mécanisme serait en fait très différent ; PHP compose les fichiers de manière _descendante_, alors que Blade les compose de manière _ascendente_. Ce serait donc au prix d'un code assez lourd.

### Définir le contenu spécifique d'une page

Dans une application, ce qui nous intéresse en premier lieu, c'est de dire comment afficher du **contenu**. Ce contenu est différent selon les pages, naturellement. Dans une application de commerce en ligne, nous voudrons une page pour afficher les listes de produits, une page pour afficher les caractéristiques d'un produit, une page sur les conditions de vente, une page pour afficher le panier du client, etc. 

Ce que nous voudrions, c'est donc un mécanisme qui nous permette de décrire _uniquement_ ces parties, sans avoir ànous préoccuper à chaque fois des parties _communes_ à toutes les pages.

Nous allons pouvoir faire cela en définissant des fragments à base de **section**. Une section, repérée par la directive `@section` est juste un conteneur qui encapsule du code HTML :

In [ ]:
%%blade

@section('menu')
    <ul>
        <li><a href= "#">Accueil</a></li>
        <li><a href= "#">Catalogue</a></li>
        <li><a href= "#">Connexion</a></li>
    </ul>
@endsection

@section('main')
    <!-- Ici le contenu principal de la page -->
@endsection

Comme on peut le voir, à chaque section est assigné un _nom_, ici `menu` ou `main`. Ces noms sont arbitraires et n'ont aucun rapport avec la syntaxe HTML.

Un fragment peut naturellement contenir autant de sections que nécessaire. En revanche, si nous adoptons cette stratégie de mise en page, les fragments ne pourront contenir _que_ des sections. Le reste ne sera pas pris en compte.

### Définir une mise en page globale

Evidemment, le fichier que nous venons d'écrire ne suffit pas, en lui-même, à afficher les informations comme nous le souhaitons. Pour cela, il nous faudrait un fichier qui contienne le format _global_ de la page. Cela, nous allons le définir dans un fichier spécifique, que les anglophones appellent un « _layout_ ». En français, cela se traduit pas _agencement_, _présentation_, ou encore _mise en page_.

Dans cette mise en page, nous allons placer les différentes sections dans des emplacements prévus pour elles dans l'espace de la page. La directive qui indique un emplacement s'appelle `@yield`.

Naturellement, nous aurons en général plusieurs sections (`@section`) et plusieurs emplacements (`@yield`). Comment faire les bonnes associations ? En se servant de l'étiquette des sections.

Voici un exemple simple de mise en page que l'on pourrait utiliser avec le fragment précédent :

In [ ]:
%%blade

<html>
    <head>
        <title>Nom du site</title>
    </head>
    <body>
        <header>
            @yield('menu')
        </header>
        <main>
            @yield('main')
        </main>
    </body>
</html>

Le nom d'une `@yield` doit correspondre au nom d'une `@section`.

Ce n'est pas tout à fait terminé, car une application peut disposer de plusieurs mises en page ! Il faut donc dire dans quel « layout » s'intègre le gabarit. On utilise pour cela la directive `@extends`, ce qui donnerait :

In [ ]:
%%blade

@extends('layouts.app')

@section('menu')
    <ul>
        <li><a href= "#">Accueil</a></li>
        <li><a href= "#">Catalogue</a></li>
        <li><a href= "#">Connexion</a></li>
    </ul>
@endsection

@section('main')
    <!-- Ici le contenu principal de la page -->
@ensection

Dans l'exemple ci-dessus, on attend un fichier `app.blade.php` dans le dossier `/resources/views/layouts`.

Admettons que le fichier du fragment s'appelle `products.blade.php`, c'est lui que l'on indiquera à la route pour la réponse :

In [ ]:
Route::get('/greeting', function () {
    return view('products');
});

On ne fait aucune référence à la mise en page globale.

### Comportements spécifiques

#### Inclusion

cette stratégie « _bottom-up_ », qui reconstruit la page à partir de données de base, peut être complétée avec une stratégie « _top_down_ » analoque à celle de PHP. Il y a pour cela plusieurs directives :
- `@include('partials.sidebar', [options => ['calendar', 'rollmap']` : similaire à PHP
- `@includeIf('partials.sidebar', [options => ['calendar', 'rollmap']` : Inclusion seulement si le gabarit existe
- `@includeWhen(true, 'partials.sidebar', [options => ['calendar', 'rollmap']` : inclusion seulement si le premier paramètre est égal à la valur booléenne `true`
- `@includeFirst(['partials.sidebar', defaults.sidebar], [options => ['calendar', 'rollmap']` : Inclusion du premier gabarit trouvé dans la liste

la combinaison de ces deux stratégies permet des mises en pages extrêmement sophistiquées ave une simplicité d'écriture assez remarquable.

#### Hiérarchies de section

La stratégie par défaut de Blade lorsqu'il rencontre un `@section` est de considérer que l'emplacement où l'insérer est _vide_. Or ce n'est pas toujours le cas. Une `@section` peut corresponde à une `@yield`, auquel cas il n'y a pas de problème, mais elle peut correspondre à une autre `@section` dans le gabarit parent. En effet, on peut trouver dans ce dernier la syntaxe suivante :

In [ ]:
%%blade

@section('events')
    <h2>Liste des événements àa venir</h2>
@show

Remarquez que la fin de section n'est pas marquée ici `@endsection`, mais `@show`.

Cela veut dire que cette section laisse la porte ouverte pour qu'un gabarit enfant vienne y ajouter du contenu. Or, la stratégie de Blade, considérant la section `events` comme vide, est de **remplacer** son contenu par ce lui de l'enfant. Pour éviter cela, il faudra ajouter une directive `@parent` dans la section de l'enfant :

In [ ]:
%%blade

@section('events')
    @parent
    
    <ul>
        @foreach($events as $e)
            <li>{{ $e }}</li>
        @endforeach
    </ul>
@endsection

## Composants


Une seconde solution, spécifique à Blade, pour résoudre le problème précédentest est la notion de **composant**, qui est proche de celle utilisée dans React et ses semblables.

### Créer un composant

En premier lieu, un composant est une classe. Celle-ci peut être créée avec `artisan`.

In [ ]:
%%bash

php artisan make:component Message

Ce faisant, `artisan` crée deux fichiers :
- Le composant lui-même : `app/View/Component/Message.php`
- Le gabarit d'affichage associé : `resources/views/components/message.blade.php`

_(avec la configuration par défaut)_


La classe de composant sert surtout à faire le rendu, via la méthode `render()`. Elle possède par ailleurs une série de méthodes utilitaires, mais qui ne sont pas essentielles pour le moment.

Dans la cas le plus simple, un composant peut juste être un fragment de HTML

In [ ]:
<!-- message.blade.php -->
<div>
    <p> Il est minuit, docteur Schweitzer !</p>
</div>

Si nous voulons importer ce composant dans une autre page, nous devons utiliser une syntaxe particulière qui est du pseudo-HTML :

In [ ]:
<x-message/>

Le préfixe `x-` indique au moteur de rendu qu'il a affaire à un composant, dont le nom est ici `message`.

### Variables de composant

Naturellement, dans la pplupart des cas, on souhaite passer au composant que l'on injecte des valeurs spécifiques, que l'on écrira :

In [ ]:
<x-message time="minuit" :name="$name" />

Dans les deux attributs ci-dessus, nous remarquons que le second, contrairement au premier est préfixé par `:`. Cela signifie à Blade que sa valeur est une expression PHP. Ici, nous voulons passer la valeur d'une variable PHP `$name` et non la chaîne de caractères "$name".

Le composant correspondant devrait être réécrit :

In [ ]:
<!-- message.blade.php -->
<div>
    <p> Il est {{ $time }}, docteur {{ $name }} !</p>
</div>

Mais on ne peut pas passer directement les valeurs au composant, il faut passer la la _classe_ de composant.

Pour cela, il faut paramétrer le constructeur de la classe `Message.php`.

In [ ]:
%%php

<?php
 
namespace App\View\Components;
 
use Illuminate\View\Component;
use Illuminate\View\View;
 
class Message extends Component
{
    /**
     * Définit les attributs ajoutés lors de l’appel au composant
     */
    public function __construct(
        public string $type,
        public string $message,
    ) {}
 
    /**
     * Indique quelle vue doit être affichée lors de l’appel du composant
     */
    public function render(): View
    {
        return view('components.alert');
    }
}

<div class="alert alert-block alert-info">
<b>Note :</b> Attention aux règles typographiques : si vous utilisez une variable comme `$lastName` (avec une majuscule), l'attribut correspondant sera `last-name` (en minuscules avec un tiret).
</div>

### Options des composants

Les composants Blade on des comportements complexes qui sont hors de la portée de ctte introduction et qui feront l'objet dun documentspécifique.